In [1]:
__author__ = 'Kayli Glidic'

# Import Library

In [3]:
#import the module
#import os
#os.environ['TSHIRT_DATA'] = '.'

from tshirt.pipeline import spec_pipeline

import matplotlib.pyplot as plt
from matplotlib import colors

%matplotlib inline

#import bokeh to enable interactive plots
from bokeh.plotting import figure
from bokeh.io import output_notebook, push_notebook, show

output_notebook()

#import yaml to read in the parameter file
import yaml

#imports to use RECTE
import os
from astropy.table import QTable
import astropy.units as u
import numpy as np
from astropy.io import fits, ascii
from astropy.table import Table, join
import pandas as pd
from astropy.time import Time


#import to copy
from copy import deepcopy

#modeling light curves
from scipy.optimize import curve_fit
import batman

#to fix errors
import pdb

#to correct for time differences
from astropy.coordinates import SkyCoord
from astropy.coordinates import EarthLocation
import time

Loading BokehJS ...

# Read in Parameter File

In [4]:
bspec = spec_pipeline.batch_spec(batchFile='corot1_batch_file.yaml')

# Import RECTE Charge Trap Correction Functions

In [5]:
import Charge_Correction_Functions
from Charge_Correction_Functions import RECTE, RECTEMulti, calculate_correction, calculate_correction_fast, charge_correction

Loading BokehJS ...

## Checking That New Ramp Profiles are the Same

In [6]:
def barycenter_correction(self):
    t1, t2 = self.get_wavebin_series()
    head = fits.getheader(self.fileL[0])
    #print("Time from tshirt: {}".format(t1['Time'][0]))
    
    expStartJD = head['EXPSTART'] + 2400000.5
    #print("Time from EXPSTART keyword {}".format(expStartJD))
    
    t1 = Time(t1['Time'][0],format='jd')
    coord = SkyCoord('06 48 19.1724141241 -03 06 07.710423478',unit=(u.hourangle,u.deg))
    loc = EarthLocation.of_site('keck')
    diff = t1.light_travel_time(coord,location=loc)
    #print('Travel Time from Keck to Barycenter= {} min'.format((diff / u.min).si))
    
    return (diff / u.day).si

In [7]:
def check_new_ramp_profiles_complex(self,nbins, median_image):
    '''
    Checks that the RECTE original ramp profiles match the New RECTE ramp profiles from altering 
    the charge_correction_fast function to be more effcient. 
    If a these new ramp profiles do not already exist this function will create them based on the given paramters.  
    
    Parameters
    ----------
    
    nbins: int
        The number of wavelength bins
        
    median_image: str
        The median image of the visit
        
    '''
    
    #read in the result file
    results = self.get_wavebin_series(nbins=nbins)
    raw_results = results[0].to_pandas()
    
    #establish the column names of the data read in 
    columns = raw_results.columns
    
    #barycenter time correction in days 
    time_correction = barycenter_correction(self)
    
    #x and y data 
    ydata = raw_results.columns[1:].values
    xdata = raw_results['Time'].values+time_correction #days correction for Solar barycenter

    table_noise = self.print_noise_wavebin(nbins=nbins)
    table_noise=table_noise.to_pandas()
    
    #parameters required for calculate_correction_fast
    exptime=100.65194699999999 #this is specific for this data on CoRoT-1b Data
    median_image = fits.getdata(median_image)
    xList_all = []
    for ind in table_noise.index: 
        Disp_st = table_noise['Disp St'][ind]
        Disp_end = table_noise['Disp End'][ind]
        Disp_xList = np.arange(Disp_st, Disp_end,1)
        xList_all.append(Disp_xList)
    
    # start time: will time how fast new ramp profiles takes to complete for loop if they don't already exist
    start = time.time()
        
    for columns,i,j in zip(ydata,xList_all,np.arange(nbins)):
        xList = i
        
        #read in optimized values for trap_pop_s and dtrap_s previously found in order to keep as many things the same as possible: we are only looking to speed up the proccess of RECTE
        #this path may be different for you!
        results_file = 'opt_result_tables/new_visit_{}_wavelength_ind_{}_nbins{}.csv'.format(self.param['nightName'],columns,nbins)
        if (os.path.exists(results_file) == True ):
            dat_result = ascii.read(results_file)
            popt = np.array(dat_result['popt'])
        else: 
            print("Path not found")
        
        #read in the correct/original RECTE ramp profiles
        #this path may be different for you!
        #notice... new_visit.... the new refers will refer to the previously saved ramp profiles with the new normalization applied
        ramp_profile_original = 'model_lightcurves/new_visit_{}_wavelength_ind_{}_nbins{}.csv'.format(self.param['nightName'],columns,nbins)
        if (os.path.exists(ramp_profile_original) == True ):
            dat_orignal = ascii.read(ramp_profile_original)
            ramp_model_original = np.array(dat_orignal['ramp_model'])
        else: 
            print("Path not found")
        
        #read in the new RECTE ramp profiles 
        ramp_profile_new = 'ramp_profiles/test_ramps_visit_{}_wavelength_ind_{}_nbins{}.csv'.format(self.param['nightName'],columns,nbins)
        if (os.path.exists(ramp_profile_new) == True ):
            dat_new = ascii.read(ramp_profile_new)
            ramp_model_new = np.array(dat_new['ramp_model'])
                
            if np.allclose(ramp_model_original,ramp_model_new,rtol=1e-15) == False:
                raise Exception("Ramp-Profiles Don't Match")
        else: 
            ramp=calculate_correction_fast(xdata,exptime,median_image,xList=xList,trap_pop_s=popt[3], dtrap_s=[popt[4]])
            ramp_model_new = np.mean(ramp,axis=0)
            dat_new = Table()
            dat_new['Time'] = xdata
            dat_new['ramp_model'] = ramp_model_new
            dat_new.write(ramp_profile_new)
            
            if np.allclose(ramp_model_original,ramp_model_new,rtol=1e-15) == False:
                raise Exception("Ramp-Profiles Don't Match")
            
    # end time
    end = time.time()

    # total time taken
    print(f"Runtime of the program is {end - start} seconds")

In [8]:
def check_new_ramp_profiles_simp(self,nbins, median_image):
    '''
    Checks that the RECTE original ramp profiles match the New RECTE ramp profiles from altering 
    the charge_correction_fast function to be more effcient. 
    If a these new ramp profiles do not already exist this function will create them based on the given paramters.  
    
    Parameters
    ----------
    
    nbins: int
        The number of wavelength bins
        
    median_image: str
        The median image of the visit
        
    '''
    
    #read in the result file
    results = self.get_wavebin_series(nbins=nbins)
    raw_results = results[0].to_pandas()
    
    #establish the column names of the data read in 
    columns = raw_results.columns
    
    #barycenter time correction in days 
    time_correction = barycenter_correction(self)
    
    #x and y data 
    ydata = raw_results.columns[1:].values
    xdata = raw_results['Time'].values+time_correction #days correction for Solar barycenter

    table_noise = self.print_noise_wavebin(nbins=nbins)
    table_noise=table_noise.to_pandas()
    
    #parameters required for calculate_correction_fast
    exptime=100.65194699999999 #this is specific for this data on CoRoT-1b Data
    median_image = fits.getdata(median_image)
    xList_all = []
    for ind in table_noise.index: 
        Disp_st = table_noise['Disp St'][ind]
        Disp_end = table_noise['Disp End'][ind]
        Disp_xList = np.arange(Disp_st, Disp_end,1)
        xList_all.append(Disp_xList)
    
    # start time: will time how fast new ramp profiles takes to complete for loop if they don't already exist
    start = time.time()
        
    for columns,i,j in zip(ydata,xList_all,np.arange(nbins)):
        xList = i
        
        #read in the correct/original RECTE ramp profiles
        #this path may be different for you!
        #notice... new_visit.... the new refers will refer to the previously saved ramp profiles with the new normalization applied
        ramp_profile_original = 'test_ramp_profiles/original_unoptimized_visit_{}_wavelength_ind_{}_nbins{}.csv'.format(self.param['nightName'],columns,nbins)
        if (os.path.exists(ramp_profile_original) == True ):
            dat_orignal = ascii.read(ramp_profile_original)
            ramp_model_original = np.array(dat_orignal['ramp_model'])
        else: 
            print("Path not found")
        
         
        ramp=calculate_correction_fast(xdata,exptime,median_image,xList=xList,trap_pop_s=200, dtrap_s=[44])
        ramp_model_new = np.mean(ramp,axis=0)
            
        if np.allclose(ramp_model_original,ramp_model_new,rtol=1e-15) == False:
            raise Exception("Ramp-Profiles Don't Match")
            
    # end time
    end = time.time()

    # total time taken
    print(f"Runtime of the program is {end - start} seconds")

In [10]:
#visit 1 ramp profile check against the original unoptimized RECTE Ramp-profile

spec_v1= bspec.return_spec_obj(ind=0)
check_new_ramp_profiles_simp(spec_v1,1, 'corot1_visit1_median_image.fits')

Runtime of the program is 0.16702818870544434 seconds


In [11]:
#visit 2 ramp profile check against the original unoptimized RECTE Ramp-profile

spec_v2= bspec.return_spec_obj(ind=1)
check_new_ramp_profiles_simp(spec_v2,1, 'corot1_visit2_median_image.fits')

Runtime of the program is 0.16757822036743164 seconds


In [12]:
#visit 3 ramp profile check against the original unoptimized RECTE Ramp-profile

spec_v3= bspec.return_spec_obj(ind=2)
check_new_ramp_profiles_simp(spec_v3,1, 'corot1_visit3_median_image.fits')

Runtime of the program is 0.16688919067382812 seconds


In [13]:
#visit 4 ramp profile check against the original unoptimized RECTE Ramp-profile

spec_v4= bspec.return_spec_obj(ind=3)
check_new_ramp_profiles_simp(spec_v4,1, 'corot1_visit4_median_image.fits')

Runtime of the program is 0.1667642593383789 seconds
